In [0]:
!pip install auto_ml

In [0]:
from auto_ml import Predictor
from auto_ml.utils_models import load_ml_model
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import numpy as np
import keras

In [0]:
import json
import os
from google.colab import drive

In [0]:
drive.mount('/content/gdrive')

# Set Kaggle Credentials

In [0]:
with open("/content/gdrive/My Drive/ML/Dev/kaggle.json") as json_data:
  kaggle_creds = json.load(json_data)

os.environ['KAGGLE_USERNAME'] = kaggle_creds["username"]
os.environ['KAGGLE_KEY'] = kaggle_creds["key"]

# Dowload Data

In [6]:
!kaggle competitions download -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.90MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 23.0MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 60.8MB/s]


# Read Data

In [0]:
df_train = pd.read_csv("/content/train.csv")

In [53]:
df_train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


# Prepare data

In [0]:
df_train = df_train.drop(["Name","Ticket","Cabin"],axis=1)

In [0]:
df_train = df_train.fillna(0)

# Build Features, Targets and Train/Test

In [0]:
msk = np.random.rand(len(df_train)) < 0.8
train = df_train[msk]
test = df_train[~msk]

In [58]:
train.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Sex', u'Age', u'SibSp',
       u'Parch', u'Fare', u'Embarked'],
      dtype='object')

In [59]:
column_descriptions = {
    'Survived': 'output',
    'Sex': 'categorical',
    'Embarked': 'categorical',
    'Name': 'ignore',
    'Ticket': 'ignore',
    'Cabin': 'ignore',
    'PassengerId' : 'ignore',
    'Pclass' : 'ignore'
}
# it is not necessary to specify numerical columns, default type

ml_predictor = Predictor(type_of_estimator='classifier', column_descriptions=column_descriptions)

ml_predictor.train(train)

test_score = ml_predictor.score(test,test.Survived)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.10
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'warm_start': True, 'learning_rate': 0.1}
Running basic data cleaning
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'warm_start': True, 'learning_rate': 0.1}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingClassifier to predict Survived
Started at:

In [0]:
file_name = ml_predictor.save("/content/model")

# Make Prediction - Print the prediction for the first 10 passengers

In [0]:
trained_model = load_ml_model(file_name)

In [0]:
py_cls = []
for label,predict in zip(test.iloc[0:10]["Survived"],trained_model.predict(test)):
  py_cls.append({"truth":label,"predict":predict})

In [70]:
py_cls

[{'predict': 0, 'truth': 0},
 {'predict': 1, 'truth': 1},
 {'predict': 0, 'truth': 1},
 {'predict': 0, 'truth': 0},
 {'predict': 1, 'truth': 1},
 {'predict': 0, 'truth': 0},
 {'predict': 1, 'truth': 1},
 {'predict': 1, 'truth': 0},
 {'predict': 0, 'truth': 0},
 {'predict': 0, 'truth': 0}]